### **Problem 3. Gaussian Elimination and Back Substitution**


Import required packages, and create matrix A and vector b

In [1]:
import numpy as np
from numpy.linalg import norm
# fix  seed  for  reproducible  result. Please  do not  change  the  seed
rng = np.random.default_rng(20232033)
n = 300
A = rng.random((n,n))
b = rng.random((n,1))
#print("A = {}; \n b = {}".format(A,b))

**3.1 (1.5/5) Gaussian Elimination (Version 0)**



**Implement your own function: TODO**

In [5]:
def gauss_elim_v0(A,b,print_flag=True):
  # check whether the calculation is valid
  assert A.shape[0] == A.shape[1],"A must be square!" # Make sure A is square
  assert A.shape[0] == b.shape[0],"Input dimension doesn't match!" # Make sure dimmension matches
  assert b.shape[1] == 1,"b is not a vector!" # Make sure b is a vector

  n = A.shape[0]
  U = np.concatenate((A, b), axis=1)

  if print_flag:
        print("Initial augmented matrix:")
        print(U)
        print("==============================================")

    # Gaussian elimination
  for k in range(n - 1):
      for j in range(k + 1, n):
          # Calculate the factor λ
          lamb = U[j, k] / U[k, k]
          # Subtract λ times the k-th row from the j-th row
          U[j, :] -= lamb * U[k, :]

          if print_flag:
              print(f"After step ({k+1}, {j+1}):")
              print(U)
              print("==============================================")
  return U

**Validate your result here using the example shown to you**

In [6]:
A_example1 = np.array([[1,-1,1],[2,-1,3],[2,0,3]],dtype=np.float64)
b_example1 = np.array([[1],[4],[5]],dtype=np.float64)

U_example1 = gauss_elim_v0(A_example1,b_example1,True)

print(U_example1)

Initial augmented matrix:
[[ 1. -1.  1.  1.]
 [ 2. -1.  3.  4.]
 [ 2.  0.  3.  5.]]
After step (1, 2):
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 2.  0.  3.  5.]]
After step (1, 3):
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  2.  1.  3.]]
After step (2, 3):
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  0. -1. -1.]]
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  0. -1. -1.]]


**3.2 (2/5) Back Substitution**

In [7]:
# this function will check whether the matrix is in row echelon form
def check_row_echelon(U):
  eps = np.finfo(np.float32).eps # updated here!!!
  # Previous version: eps = np.finfo(np.float64).eps
  test = np.tril(U,-1)
  #print(test) np.abs(test)
  return np.all(abs(test)<=eps)


**Implement your own function: TODO**

In [23]:
def back_subs(U, print_flag=True):
    assert check_row_echelon(U), "U must be in row echelon form"

    n = U.shape[0]
    x = np.zeros((n, 1))

    max_abs_x = 1.0  # Initialize the maximum absolute value of x

    for i in range(n - 1, -1, -1):
        if U[i, i] == 0:
            x[i] = np.nan  # Handle division by zero
        else:
            # Calculate x[i], handling potential overflow
            try:
                x[i] = (U[i, -1] - np.sum(U[i, i + 1:-1] * x[i + 1:])) / U[i, i]
                max_abs_x = max(max_abs_x, np.abs(x[i]))
            except OverflowError:
                x[i] = np.nan  # Handle overflow

        # Normalize x if its elements become too large
        if max_abs_x > 1e10:
            x /= max_abs_x
            max_abs_x = 1.0  # Reset the maximum absolute value

        if print_flag:
            print(f"x{i} = {x[i]}")

    return x

Validate your result here using the example shown to you

In [24]:
x_example1 = back_subs(U_example1,True)
print(x_example1)

x2 = [1.]
x1 = [1.]
x0 = [1.]
[[1.]
 [1.]
 [1.]]


**Call your own functions and validate your solutions (x1) by using relative distance ||Ax1 - b||/||A||: TODO**

In [25]:
def my_solver_v0(A, b):
    U = gauss_elim_v0(A, b, False)
    x = back_subs(U, False)
    return x

Validate your result here

In [28]:
# Call my_solver_v0 to solve the linear system
x1 = my_solver_v0(A, b)

# Normalize the solution vector x1
max_abs_x1 = np.max(np.abs(x1))
x1_normalized = x1 / max_abs_x1

# Normalize the matrix A
max_abs_A = np.max(np.abs(A))
A_normalized = A / max_abs_A

# Calculate the relative error
dist = np.linalg.norm(np.dot(A_normalized, x1_normalized) - b) / np.linalg.norm(A_normalized, ord='fro')

print("The relative error ||Ax1-b||/||A|| = {}".format(dist))

The relative error ||Ax1-b||/||A|| = 0.10664110135915907


**Call the numpy built-in function to solve the given questions (solution is x2), and compare it with your result using relative distance ||x1-x2||/||x2||: TODO**

In [29]:
# Call numpy.linalg.solve to solve the linear system
x2 = np.linalg.solve(A, b)

# Calculate the relative distance between x1 and x2
dist = np.linalg.norm(x1 - x2) / np.linalg.norm(x2)

print("The relative distance ||x1-x2||/||x2|| = {}".format(dist))

The relative distance ||x1-x2||/||x2|| = 974.2380471493556


**3.3 (1.5/5) Gaussian Elimination (Version 1)**

In [30]:
# Please run this cell before running the cells below! This alters the matrix so that we need
# the more complete version of the gaussian elimination algorithm
A[0,0] = 0

**Implement your own function: TODO**

In [31]:
def gauss_elim_v1(A, b, print_flag):
    # check whether the calculation is valid
    assert A.shape[0] == A.shape[1], "A must be square" # Make sure A is square
    assert A.shape[0] == b.shape[0], "Input dimension doesn't match" # Make sure dimension matches
    assert b.shape[1] == 1, "b is not a vector" # Make sure b is a vector

    n = A.shape[0]
    U = np.concatenate((A, b), axis=1)  # Generate the augmented matrix U by concatenating A and b

    for k in range(n - 1):  # Iterate over rows of U
        # Find the index of the row with the largest absolute value in the k-th column
        i = np.argmax(np.abs(U[k:, k])) + k
        # Exchange rows k and i to bring the largest element in magnitude to the top
        U[[k, i]] = U[[i, k]]

        if print_flag:
            print("Intermediate augmented matrix after row exchange:")
            print(U)
            print("==============================================")

        for j in range(k + 1, n):  # Eliminate rows below using the current row
            λ = U[j, k] / U[k, k]  # Calculate the multiplier
            U[j] -= λ * U[k]  # Subtract λ times the k-th row from the j-th row

            if print_flag:
                print("Intermediate augmented matrix after elimination step:")
                print(U)
                print("==============================================")

    return U

In [32]:
def my_solver_v1(A, b):
    U = gauss_elim_v1(A, b, False)  # Perform Gaussian elimination
    x = back_subs(U, False)  # Perform backward substitution
    return x

**Validate your result here using the example shown to you**

In [33]:
A_example2 = np.array([[0,1,1],[2,6,4],[1,1,4]],dtype=np.float64)
b_example2 = np.array([[-1],[6],[9]],dtype=np.float64)

U_example2 = gauss_elim_v1(A_example2,b_example2,True)

print(U_example2)

Intermediate augmented matrix after row exchange:
[[ 2.  6.  4.  6.]
 [ 0.  1.  1. -1.]
 [ 1.  1.  4.  9.]]
Intermediate augmented matrix after elimination step:
[[ 2.  6.  4.  6.]
 [ 0.  1.  1. -1.]
 [ 1.  1.  4.  9.]]
Intermediate augmented matrix after elimination step:
[[ 2.  6.  4.  6.]
 [ 0.  1.  1. -1.]
 [ 0. -2.  2.  6.]]
Intermediate augmented matrix after row exchange:
[[ 2.  6.  4.  6.]
 [ 0. -2.  2.  6.]
 [ 0.  1.  1. -1.]]
Intermediate augmented matrix after elimination step:
[[ 2.  6.  4.  6.]
 [ 0. -2.  2.  6.]
 [ 0.  0.  2.  2.]]
[[ 2.  6.  4.  6.]
 [ 0. -2.  2.  6.]
 [ 0.  0.  2.  2.]]


**Call your own functions and validate your solutions (x1) by using relative distance ||Ax1 - b||/||A||: TODO**

In [35]:
# Call my_solver_v1 to solve the linear system
x1 = my_solver_v1(A, b)

# Calculate the relative error ||Ax1 - b|| / ||A||_F
dist = np.linalg.norm(np.dot(A, x1) - b) / np.linalg.norm(A, 'fro')

print("The relative error ||Ax1 - b|| / ||A||_F = {}".format(dist))

The relative error ||Ax1 - b|| / ||A||_F = 0.04047798257991626


**Call the numpy built-in function to solve the given questions (solution is x2), and compare it with your result using relative distance ||x1-x2||/||x2||: TODO**

In [36]:
# Call numpy.linalg.solve to solve the linear system
x2 = np.linalg.solve(A, b)

# Calculate the relative distance between x1 and x2
dist = np.linalg.norm(x1 - x2) / np.linalg.norm(x2)

print("The relative distance ||x1 - x2|| / ||x2|| = {}".format(dist))

The relative distance ||x1 - x2|| / ||x2|| = 0.997147236119322
